# IR Assignment


In [1]:
#Importing all the required libraries
import numpy as np
import os
import re
import operator
import glob
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import string
import copy
import pandas as pd
import pickle
import nltk

In [2]:
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

# Question 1:

## Jaccard Coefficient
<br>
The goal is to find the Jaccard coefficient between a given query and the document. The formula used is
mentioned below as:
<br>
Jaccard Coefficient = Intersection of (doc,query) / Union of (doc,query)
<br>
The high the value of the Jaccard coefficient, the more the document is relevant for the query.

1. Use the same data given in assignment 1 and carry out the same preprocessing steps as mentioned
before.
2. To calculate this make set of the document token and query token and perform intersection and union
between the query and each document.
3. Report the top 5 relevant documents based on the value of the Jaccard coefficient.

In [3]:
lemmatizer = WordNetLemmatizer() 
stopWords = set(stopwords.words('english'))

1. Preprocessing steps as mentioned before.

In [4]:
# PreProcessing Steps

def preProcess(file):
    # Convert file to lowercase.
    file = file.lower()
    
    # Word Tokenization
    file = word_tokenize(file)
    
    # Removing stopwords
    token_list = []
    for word in file:
        if word not in stopWords:
            token_list.append(word)
    
    # Remove punctuations.
    tokens = [word.translate(str.maketrans('', '', '\t')) for word in token_list]

    punctuations = (string.punctuation).replace("'", "")
    trans_table = str.maketrans('', '', punctuations)
    stripped_words = [word.translate(trans_table) for word in tokens]
    tokens = [str for str in stripped_words if str]
    
    #Removing blank tokens
    while("" in tokens) :
        tokens.remove("")
 
    return tokens

In [5]:
path = r"C:\Users\jsk\Desktop\IIITD\SEMESTER 2\IR\Assignment\Files"
files = os.listdir(path)

In [6]:
files

['1st_aid.txt',
 'a-team',
 'abbott.txt',
 'aboutada.txt',
 'acetab1.txt',
 'aclamt.txt',
 'acne1.txt',
 'acronym.lis',
 'acronym.txt',
 'acronyms.txt',
 'adameve.hum',
 'adcopy.hum',
 'addrmeri.txt',
 'admin.txt',
 'adrian_e.faq',
 'ads.txt',
 'adt_miam.txt',
 'advrtize.txt',
 'aeonint.txt',
 'age.txt',
 'aggie.txt',
 'aids.txt',
 'airlines',
 'alabama.txt',
 'alcatax.txt',
 'alcohol.hum',
 'alflog.txt',
 'allfam.epi',
 'allusion',
 'all_grai',
 'amazing.epi',
 'ambrose.bie',
 'amchap2.txt',
 'analogy.hum',
 'aniherb.txt',
 'anime.cli',
 'anime.lif',
 'anim_lif.txt',
 'annoy.fascist',
 'anorexia.txt',
 'answers',
 'anthropo.stu',
 'antibiot.txt',
 'antimead.bev',
 'aphrodis.txt',
 'appbred.brd',
 'appetiz.rcp',
 'applepie.des',
 'apsaucke.des',
 'apsnet.txt',
 'arab.dic',
 'arcadian.txt',
 'argotdic.txt',
 'arnold.txt',
 'art-fart.hum',
 'arthriti.txt',
 'ateam.epi',
 'atherosc.txt',
 'atombomb.hum',
 'att.txt',
 'aussie.lng',
 'avengers.lis',
 'awespinh.sal',
 'ayurved.txt',
 'a_fish

2. To calculate this make set of the document token and query token and perform intersection and union between the query and each document

In [7]:
# Function to calculate Jaccard Coefficient
def calculate_jaccard(query, document):
    query = preProcess(query)
    doc = preProcess(document)
    intersections = len(set(query)&set(doc))    # Finding the intersections between query and document
    unions = len(set(query)|set(doc))           # Finding the unions between query and document
    jaccard = intersections/unions              # Calculate Jaccard coefficient
    return jaccard

In [12]:
# Retreiving Top 5 Documents using Jaccard Coefficient

def jaccard_retrieval_docs(query,files):
    docs_Dict={}                       
    jaccard_coefficients_list=[]           # List of jaccard coefficient for each doc
    i=0
    for file in files:
        docs_Dict[i]=file
        try:
            f = open("./Files/"+file, "r")
            data = f.read()
        except:
            f = open("./Files/"+file, "rb")
            data = f.read().decode('utf-8', 'backslashreplace')
        jaccard_coefficients_list.append(calculate_jaccard(query,data))      
        i+=1
    jaccard_coefficients_list = np.array(jaccard_coefficients_list)
    top_5_docs = jaccard_coefficients_list.argsort()[-5:][::-1]       # Finding the top 5 documents from the end.
    for i in range(len(top_5_docs)):
        print("Jaccard Coefficient: ", jaccard_coefficients_list[top_5_docs[i]], ", Document: " ,docs_Dict[top_5_docs[i]])

In [9]:
def queryInput():
    q = input("Enter query: ")
    q = preProcess(q)
    print(q)
    return q

In [10]:
query=input()

I am sherlock


3. Report the top 5 relevant documents based on the value of the Jaccard coefficient.

In [11]:
jaccard_retrieval_docs(query,files)

Jaccard Coefficient:  0.005780346820809248 , Document:  livnware.hum
Jaccard Coefficient:  0.0016863406408094434 , Document:  subb_lis.txt
Jaccard Coefficient:  0.0016339869281045752 , Document:  mov_rail.txt
Jaccard Coefficient:  0.0005605381165919282 , Document:  crzycred.lst
Jaccard Coefficient:  0.0005125576627370579 , Document:  insults1.txt


## TF-IDF Matrix 

The goal is to generate a TF-IDF matrix for each word in the vocab and obtain a TF-IDF score for a
given query. TF-IDF has two parts Term Frequency and Inverse Document Frequency.
<br><br>

1. Computing Term Frequency involves calculating the raw count of the word in each document and
stored as a nested dictionary for each document.
2. To calculate the document frequency of each word, find the postings list of each word and subsequently
find the no. of documents in each posting list of each word.
3. The IDF value of each word is calculated using the formula as mention below:
Using smoothing:-
IDF(word)=log(total no. of documents/document frequency(word)+1)
4. The Term Frequency is calculated using 5 different variants:
<br> 


In [13]:
doc = 0
tokens= []
ID_File_Mapping = []
 
# Reading each file and creating tokens for each file
for file in os.listdir('./Files'):
    try:
        f = open("./Files/"+file, "r") 
        text = " ".join(f.read().split()) 
    except:
        f = open("./Files/"+file, "rb")      
        text = f.read().decode('utf-8', 'backslashreplace')
    f.close()

    name = os.path.basename(file)
    ID_File_Mapping.append([name])      # ID- Name Mapping of the file
    
#     print(text)
    tokens.append(preProcess(text))  
    
ID_File_Mapping = pd.DataFrame(ID_File_Mapping)

In [14]:
len(tokens)

1134

### Calculating Document Frequency

In [15]:
#Document Frequency - no. of documents in each posting list of each word.
doc_frequency = {}   
for t in tokens:
    words = list(set(t))
    for w in words:
        if doc_frequency.get(w) == None:
            doc_frequency[w] = 1   # If new word found, count=1
        else:
            count = doc_frequency.get(w)
            count += 1              #else increase the frequency
            doc_frequency[w] = count

In [16]:
print(len(doc_frequency))
total_docs = len(files)
print('Total Number of documents are',total_docs)

84028
Total Number of documents are 1134


#### The IDF value of each word is calculated using the formula as mention below: Using smoothing:- IDF(word)=log(total no. of documents/document frequency(word)+1)

In [17]:
#Inverse Document Frequency IDF(word)=log(total no. of documents/document frequency(word)+1)
IDF = {}
index = doc_frequency.keys()
for i in index:
    freq = doc_frequency[i]   #document frequency of each word in document frquency is picked
    inv_doc_freq = np.log(total_docs/freq +1)
    IDF[i] = inv_doc_freq

In [18]:
#Function block for Calculating tf-idf value---tf*idf as tf=idf will be calculated based on all the 5 variants
def tf_idf_score(tf):
    tf_idf = []  #creating list of list of tf-idf
    for score in tf:
        tf_idf_dict = {}
        indexes = score.keys()
        for w in indexes:
            tf_idf_val = score[w] * IDF[w]                #Multiplying tfidf=tf*idf
            tf_idf_dict[w] = tf_idf_val
        tf_idf.append(tf_idf_dict)
    return tf_idf

In [19]:
#fetching rank document based on tfidf

def tfidf_rank_doc(tfidf_matrix, query):
    
    query_res = [0.0] * len(total_indexes)
    res = [i for i, q in enumerate(total_indexes) if q in set(query)]
    for i in range(len(total_indexes)):
        if i in res: 
            query_res[i]=1.0     
        else: 
            query_res[i]=0.0
        
    tf_docs = {}
    for i in range(total_docs):
        tf_docs[i] = 0.0
    for r in res:
        count = -1
        for v in tfidf_matrix:
            count += 1
            tf_docs[count] += v[r]
    tf_docs = dict(sorted(tf_docs.items(),key=operator.itemgetter(1),reverse=True))   

    return tf_docs, query_res

In [25]:
# Function to retrive top 5 documents based on variant matrix
def Retrieve_Top_5_Docs(tfidf_matrix ,query, var):
    tf_docs , query_res= tfidf_rank_doc(tfidf_matrix, query)
    count = -1
    temp = 0
    final = []
    for item in tf_docs.keys():
        count += 1
        if count == 5:
            print('Top 5 Documents based on ',var ,' tf-idf are ',final)
            for i in final:
                print("TF_IDF Score: ", tf_docs[i], "Document: ",ID_File_Mapping.iloc[i][0])
            temp = 1
            break
        final.append(item)
    if temp == 0:
        print('Top 5 Documents based on ',var ,' tf-idf are :',final)    
        for i in final:
            print("TF_IDF Score: ", tf_docs[i], "Document: ",ID_File_Mapping.iloc[i][0])
            
    return query_res         

In [21]:
total_indexes = list(doc_frequency.keys())
len(total_indexes)

84028

In [22]:
raw_word_count = []   #list for storing word count for each doc
for item in tokens:
    raw_word_doc ={}      
    for w in item:
        if raw_word_doc.get(w) == None:  
            raw_word_doc[w] = 1       # If new word found, count=1
        else:
            count = raw_word_doc.get(w)
            count += 1               #else increase the count
            raw_word_doc[w] = count
    raw_word_count.append(raw_word_doc)         

## 1. Binary Variant - 0,1


In [27]:
# Term Frequency is calculated using 5 different variants:

binary_frequency = []   
for word in raw_word_count:
    binary_dict = {}        
    indexes = word.keys()
    for i in indexes:
        if(word.get(i) > 0):
            bool_freq = 1
        else: 
            bool_freq = 0             
        binary_dict[i] = bool_freq
        
    binary_frequency.append(binary_dict)

binary_tf_idf_score = tf_idf_score(binary_frequency)

#tf-idf matrix using binary
binary_matrix = []   
for score in binary_tf_idf_score:
    bin_doc = []
    for s in total_indexes:
        if s in score :            
            bin_doc.append(score[s])   #if word present then store tfidf
        else:
            bin_doc.append(0.0)    #else 0
    binary_matrix += [bin_doc]   

In [28]:
q = queryInput()  
binary_query = Retrieve_Top_5_Docs(binary_matrix,q,'Binary')

Enter query: A simple device
['simple', 'device']
Top 5 Documents based on  Binary  tf-idf are  [9, 29, 35, 140, 173]
TF_IDF Score:  4.884552500454063 Document:  acronyms.txt
TF_IDF Score:  4.884552500454063 Document:  all_grai
TF_IDF Score:  4.884552500454063 Document:  anime.cli
TF_IDF Score:  4.884552500454063 Document:  bored.txt
TF_IDF Score:  4.884552500454063 Document:  bw.txt


### Pros
1. Binary word count is simple and fast to create. <br>
2. Process large documents collections quickly. Takes only O(n) time.


### Cons
1. Treat every word equally. And due to this it does not take into account that some characters occuring more frequently might provide some info but this is not captured in 0-1
2. With the increase in the vocab size, length of vectors increase too which makes this variant slow.
 

## 2. Raw Word Count - f(t,d)

In [32]:
raw_tf_idf_score = tf_idf_score(raw_word_count)

#tf-idf score using raw count
raw_matrix = []   
for word in raw_tf_idf_score:
    raw_docs = []
    for w in total_indexes:
        if w in word :
            raw_docs.append(word[w])   #if word present then store tfidf
        else:
            raw_docs.append(0.0)     #else 0
    raw_matrix += [raw_docs]    

In [34]:
q = queryInput()  
raw_query = Retrieve_Top_5_Docs(raw_matrix,q,'Raw Count')

Enter query: A simple device
['simple', 'device']
Top 5 Documents based on  Raw Count  tf-idf are  [9, 1069, 501, 113, 500]
TF_IDF Score:  39.94560674024448 Document:  acronyms.txt
TF_IDF Score:  34.45897006036514 Document:  variety3.asc
TF_IDF Score:  33.589783323753174 Document:  humor9.txt
TF_IDF Score:  32.919820079276015 Document:  bimg.prn
TF_IDF Score:  32.919820079276015 Document:  humatran.jok


### Pros
1. Easy to compute
2. Used to extract the most descriptive terms in a document

### Cons
1. It is slow for large dictionaries as it directly computes word count space  for document similarity.
2. it does not capture semantics, position in text, co-occurrences in different documents

##  3. Term Frequency - f(t,d)/Pf(t‘, d)

In [35]:
term_frequency = []     #list of log normalization term frequency
for word in raw_word_count:
    term_dict = {}        
    indexes = word.keys()
    term_sum = sum(word.values())     #counting no. of tokens in each document
    for i in indexes:
        term_dict[i] = (word.get(i) / term_sum)
    term_frequency.append(term_dict)
    
term_tf_idf_score = tf_idf_score(term_frequency)

##tf-idf score using term frequency
term_matrix = []  
for score in term_tf_idf_score:
    term_docs = []
    for i in total_indexes:
        if i in score :
            term_docs.append(score[i])    #if word present then store tfidf
        else:
            term_docs.append(0.0)    #else 0
    term_matrix += [term_docs]   

In [36]:
q = queryInput()  

term_query = Retrieve_Top_5_Docs(term_matrix,q,'Term Frequency')

Enter query: A simple device
['simple', 'device']
Top 5 Documents based on  Term Frequency  tf-idf are  [500, 499, 58, 798, 113]
TF_IDF Score:  0.04308876973727227 Document:  humatran.jok
TF_IDF Score:  0.04191180797130048 Document:  humatra.txt
TF_IDF Score:  0.03305202819204419 Document:  atombomb.hum
TF_IDF Score:  0.026112611713590195 Document:  poli.tics
TF_IDF Score:  0.02600301744018643 Document:  bimg.prn


### Pros
1. When number of times occurences of word is more important than the word. We use Term frquency.
2. Helps in extracting the most descriptive word in a document.

### Cons
1. It cannot extracts semantic similiarities.
2. It can suffer from memory inefficiency for large document collections


## 4. Log Normalisation = log(1 +f(t,d)))

In [37]:
log_frequency = []      
for word in raw_word_count:
    log_dict = {}         #dictionary for log normalization term frequency
    indexes = word.keys()
    for i in indexes:    
        freq = word.get(i)
        log_dict[i] = np.log(1 + freq)     # log(1 +f(t,d)))
    log_frequency.append(log_dict)
    
    
log_tf_idf_score = tf_idf_score(log_frequency)

log_matrix = []  
for score in log_tf_idf_score:
    log_docs = []
    for s in total_indexes:
        if s in score :            
            log_docs.append(score[s])    #if word present then store tfidf
        else:
            log_docs.append(0.0)    #else 0
    log_matrix += [log_docs]  

In [38]:
q = queryInput()  
log_query = Retrieve_Top_5_Docs(log_matrix,q,'Logarithmic')   

Enter query: A simple device
['simple', 'device']
Top 5 Documents based on  Logarithmic  tf-idf are  [501, 35, 9, 1069, 451]
TF_IDF Score:  10.043023323092292 Document:  humor9.txt
TF_IDF Score:  9.86426306290889 Document:  anime.cli
TF_IDF Score:  9.592160534142803 Document:  acronyms.txt
TF_IDF Score:  9.169276147058518 Document:  variety3.asc
TF_IDF Score:  9.1199728654888 Document:  hackmorality.txt


### Pros
1. When the occurences of the term is required as an important feature then log weighting is used. For example a word occuring 20 times in a document is more relevant but not 20 times more relevant. So here we will use log normalization to calculate term frequency.
2. It prevent bias in term frequency from terms in shorter or longer documents

### Cons
1. It also does not help in capturing semantic structures ovee other documents 
2. Can gives more importance to less important but more occurrence word.


## 5. Double Normalisation Frequency = 0.5 + ((0.5*tf)/max(tf))

In [39]:
double_frequency = []  #list for storing double normalization term frequency for each doc


for word in raw_word_count:
    double_dict = {}             #dictionary for storing double normalization term frequency
    indexes = word.keys()
    for i in indexes:
        double_dict[i] = 0.5 + (0.5*(word.get(i) / max(word.values())))    #Double Normalisation Frequency = 0.5 + ((0.5*tf)/max(tf))
    double_frequency.append(double_dict)
    
double_tf_idf_score = tf_idf_score(double_frequency)

#tf-idf score using double
double_matrix = []    
for score in double_tf_idf_score:
    double_docs = []
    for s in total_indexes:
        if s in score :
            double_docs.append(score[s])    #if word present then store tfidf
        else:
            double_docs.append(0.0)    #else 0
    double_matrix += [double_docs]   

In [40]:
q = queryInput()  

double_query = Retrieve_Top_5_Docs(double_matrix,q,'Double') 

Enter query: A simple device
['simple', 'device']
Top 5 Documents based on  Double  tf-idf are  [35, 751, 1069, 9, 423]
TF_IDF Score:  2.7189941953294836 Document:  anime.cli
TF_IDF Score:  2.6835136402660544 Document:  onan.txt
TF_IDF Score:  2.6473891672530145 Document:  variety3.asc
TF_IDF Score:  2.5935853666673516 Document:  acronyms.txt
TF_IDF Score:  2.5788857035632096 Document:  gd_ol.txt


### Pros

1. Useful when one document is very large and word occurences is just due to large voab and not its relevancy. It normalize the tf weights of all terms occurring in a document by the maximum tf in that document.
2. It prevents a bias towards longer document by normalizing the raw frequency of the term into a comparable scale.

### Cons

1. The method is unstable and hard to tune because a change in the stop word list can dramatically alter term weightings and ranking too.<br>
2. Outlier term which is occuring large number of times in a document may become representative of the content of that document.

## Compute the TF-IDF score for the query using the TF-IDF matrix. 
 
### Report the top 5 relevant documents based on the score.

1. Binary TFIDF

Top 5 Documents based on  Binary  tf-idf are  [9, 29, 35, 140, 173]
<br><br>
TF_IDF Score:  4.884552500454063 Document:  acronyms.txt
<br>
TF_IDF Score:  4.884552500454063 Document:  all_grai
<br>
TF_IDF Score:  4.884552500454063 Document:  anime.cli
<br>
TF_IDF Score:  4.884552500454063 Document:  bored.txt
<br>
TF_IDF Score:  4.884552500454063 Document:  bw.txt

2. Raw Word  Count TF=iDF

Top 5 Documents based on  Raw Count  tf-idf are  [9, 1069, 501, 113, 500]<br><br>
TF_IDF Score:  39.94560674024448 Document:  acronyms.txt<br>
TF_IDF Score:  34.45897006036514 Document:  variety3.asc<br>
TF_IDF Score:  33.589783323753174 Document:  humor9.txt<br>
TF_IDF Score:  32.919820079276015 Document:  bimg.prn<br>
TF_IDF Score:  32.919820079276015 Document:  humatran.jok<br>

3. Term Frequency TF-IDF

Top 5 Documents based on  Term Frequency  tf-idf are  [500, 499, 58, 798, 113]<br><br>
TF_IDF Score:  0.04308876973727227 Document:  humatran.jok<br>
TF_IDF Score:  0.04191180797130048 Document:  humatra.txt<br>
TF_IDF Score:  0.03305202819204419 Document:  atombomb.hum<br>
TF_IDF Score:  0.026112611713590195 Document:  poli.tics<br>
TF_IDF Score:  0.02600301744018643 Document:  bimg.prn<br>

4. Lograithmicc TF-IDF

Top 5 Documents based on  Logarithmic  tf-idf are  [501, 35, 9, 1069, 451]<br><br>
TF_IDF Score:  10.043023323092292 Document:  humor9.txt<br>
TF_IDF Score:  9.86426306290889 Document:  anime.cli<br>
TF_IDF Score:  9.592160534142803 Document:  acronyms.txt<br>
TF_IDF Score:  9.169276147058518 Document:  variety3.asc<br>
TF_IDF Score:  9.1199728654888 Document:  hackmorality.txt<br>

5. Double TF-IDF

Top 5 Documents based on  Double  tf-idf are  [35, 751, 1069, 9, 423]<br><br>
TF_IDF Score:  2.7189941953294836 Document:  anime.cli<br>
TF_IDF Score:  2.6835136402660544 Document:  onan.txt<br>
TF_IDF Score:  2.6473891672530145 Document:  variety3.asc<br>
TF_IDF Score:  2.5935853666673516 Document:  acronyms.txt<br>
TF_IDF Score:  2.5788857035632096 Document:  gd_ol.txt<br>